<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import sqlalchemy
import plotly.express as px

In [3]:
# Используем SQLAlchemy для обёртки соединения, чтобы убрать "UserWarning:"
connection = sqlalchemy.create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}")

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [4]:
# текст запроса
query_3_1 = f'''select
                count(id)
                from public.vacancies
'''

In [5]:
# результат запроса
count_vacancies = pd.read_sql_query(query_3_1, connection).loc[0, 'count']
print(f"Вакансий в базе: {count_vacancies}")

Вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [6]:
# текст запроса
query_3_2 = f'''select
                count(id)
                from public.employers
'''

In [7]:
# результат запроса
count_employers = pd.read_sql_query(query_3_2, connection).loc[0, 'count']
print(f"Работадателей в базе: {count_employers}")

Работадателей в базе: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [8]:
# текст запроса
query_3_3 = f'''select
                count(id)
                from public.areas
'''

In [9]:
# результат запроса
count_areas = pd.read_sql_query(query_3_3, connection).loc[0, 'count']
print(f"Регионов в базе: {count_areas}")

Регионов в базе: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [10]:
# текст запроса
query_3_4 = f'''select
                count(id)
                from public.industries
'''

In [11]:
# результат запроса
count_industries = pd.read_sql_query(query_3_4, connection).loc[0, 'count']
print(f"Сфер деятельности в базе: {count_industries}")

Сфер деятельности в базе: 294


***

***Выводы по предварительному анализу данных:***

В базе данных представлены 49 197 вакансий в 1 362 регионах от 23 501 работодателей, осуществляющих свою деятельность в 294 сферах.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [12]:
# текст запроса
query_4_1 = f'''select
                a.name as "Город",
                count(v.id) as "Кол-во вакансий"
                from public.vacancies v
                join public.areas a on v.area_id=a.id
                group by a.id
                order by count(v.id) desc
'''

In [13]:
# результат запроса
area_df = pd.read_sql_query(query_4_1, connection)
# выводим пятёрку лидеров по количеству вакансий
display(area_df.head(5))

,Город,Кол-во вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [14]:
# текст запроса
query_4_2 = f'''select
                count(id)
                from public.vacancies
                where salary_from is not null
                    or salary_to is not null
'''

In [15]:
# результат запроса
count_salary = pd.read_sql_query(query_4_2, connection).loc[0, 'count']
print(f"У {count_salary} ({round(count_salary/count_vacancies*100)}%) \
вакансий заполнено хотя бы одно из двух полей с зарплатой.")

У 24073 (49%) вакансий заполнено хотя бы одно из двух полей с зарплатой.


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [16]:
# текст запроса
query_4_3 = f'''select
                avg(salary_from) as "Среднее нижней границы",
                avg(salary_to) as "Среднее верхней границы"
                from public.vacancies
'''

In [17]:
# результат запроса
avg_salary = pd.read_sql_query(query_4_3, connection)
print(f"Среднее значение для нижней границы зарплатной вилки \
{round(avg_salary.loc[0, 'Среднее нижней границы'])}, для верхней - \
{round(avg_salary.loc[0, 'Среднее верхней границы'])}.")


Среднее значение для нижней границы зарплатной вилки 71065, для верхней - 110537.


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [18]:
# текст запроса
query_4_4 = f'''select
                -- Сочетание строк schedule и employment объединим в один столбец через "/"
                schedule || ' / ' || employment as "Тип рабочего графика / трудоустройства",
                count(id) as "Кол-во вакансий"
                from public.vacancies
                group by schedule, employment
                order by count(id) desc
'''

In [19]:
# результат запроса
schedule_and_employment = pd.read_sql_query(query_4_4, connection)
display(schedule_and_employment)

,Тип рабочего графика / трудоустрой,Кол-во вакансий
0,Полный день / Полная занятость,35367
1,Удаленная работа / Полная занятость,7802
2,Гибкий график / Полная занятость,1593
3,Удаленная работа / Частичная занятость,1312
4,Сменный график / Полная занятость,940
5,Полный день / Стажировка,569
6,Вахтовый метод / Полная занятость,367
7,Полный день / Частичная занятость,347
8,Гибкий график / Частичная занятость,312
9,Полный день / Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [20]:
# текст запроса
query_4_5 = f'''select
                experience as "Требуемый опыт работы",
                count(id) as "Кол-во вакансий"
                from public.vacancies
                group by experience
                order by count(id)
'''

In [21]:
# результат запроса
experience = pd.read_sql_query(query_4_5, connection)
display(experience)

# Строим круговую диаграмму по полученным данным
fig = px.pie(
    data_frame=experience, # датафрейм
    values='Кол-во вакансий', # значения
    names='Требуемый опыт работы', # имена категорий
    color_discrete_sequence=['PaleGreen', 'PaleTurquoise', 'Bisque', 'Lavender'], # цвета
    title='Распределение вакансий по требуемому опыту работы' # заголовок
)

fig.update_traces(
    textinfo='label+percent', # выводим имена категорий и процент в метках данных
)
fig.update(layout_showlegend=False) # удаляем легенду

fig.show()

,Требуемый опыт работы,Кол-во вакансий
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

![](plots/experience_total.png)

***Выводы по детальному анализу вакансий:***

Наибольшее количество вакансий представлено в Москве, на втором и третьем месте Санкт-Петербург и Минск. Для указанных городов характерна большая численность население и наличие множества учебных заведений в том числе крупнейших в СНГ, что в купе с миграционным притоком населения из регионов, создаёт предложение на рынке труда квалифицированных IT-специалистов и как следствие спрос, что выражается в количестве вакансий.

Среднее значение для нижней границы зарплатной вилки 71065, для верхней - 110537. При этом данные поля заполнены только в половине вакансий (24073 (49%)).

В более чем 90% вакансий в качестве типа трудоустройства указана полная занятость, для 72% характерна комбинация "Полный день" / "Полная занятость". В целом IT-вакансии предполагают классическую пятидневку, для них в большинстве случаев не характерен вахтовый метод и сменный график, и при этом имеет место удалённая работа.

По распределению желаемого опыта работы можно сделать вывод, что больше половины работодателей считают, что опыта работы от 1 до 3 лет достаточно для выполнения обязанностей. Соискатели с данным опытом работы уже обладают некоторыми начальными навыками для выполнения задач, и тратят меньше времени на обучение, чем люди без опыта, при этом у них ещё нет требований к высокой заработной плате, как у опытных специалистов.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [22]:
# текст запроса
query_5_1 = f'''-- Создаём таблицу, где будет первое место
                (select
                e.name,
                count(v.id)
                from public.vacancies as v
                join public.employers as e on v.employer_id = e.id
                group by e.id
                order by count(v.id) desc
                limit 1)
                -- Объединяем с таблицей с пятым местом
                union all
                (select
                e.name,
                count(v.id)
                from public.vacancies as v
                join public.employers as e on v.employer_id = e.id
                group by e.id
                order by count(v.id) desc
                offset 4 limit 1)
'''

In [23]:
# результат запроса
emp = pd.read_sql_query(query_5_1, connection)
print(f"На первом месте по количеству вакансий находится {emp.loc[0, 'name']} ({emp.loc[0, 'count']} \
вакансий), на пятом - {emp.loc[1, 'name']} ({emp.loc[1, 'count']} вакансий).")

На первом месте по количеству вакансий находится Яндекс (1933 вакансий), на пятом - Газпром нефть (331 вакансий).


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [24]:
# текст запроса
query_5_2 = f'''select
                a.name as "Регион",
                count(e.id) as "Кол-во работадателей",
                count(v.id) as "Кол-во вакансий"
                -- Т.к. важна информация о регионах без вакансий или работадателей,
                -- используем LEFT JOIN.
                from public.areas as a
                left join public.employers as e on a.id = e.area
                left join public.vacancies as v on a.id = v.area_id
                group by a.id
                order by count(v.id), count(e.id) desc
'''

In [25]:
# результат запроса
employers_area = pd.read_sql_query(query_5_2, connection)
display(employers_area.head(10))

,Регион,Кол-во работадателей,Кол-во вакансий
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Беларусь,18,0
5,Ростовская область,18,0
6,Азербайджан,17,0
7,Республика Татарстан,16,0
8,Нижегородская область,16,0
9,Узбекистан,15,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [26]:
# текст запроса
query_5_3 = f'''select
                e.name as "Работодатель",
                count(distinct v.area_id) as "Кол-во регионов с вакансиями"
                from public.vacancies as v
                join public.employers as e on v.employer_id = e.id
                group by e.id
                order by count(distinct v.area_id) desc
'''

In [27]:
# результат запроса
employers_area = pd.read_sql_query(query_5_3, connection)
display(employers_area.head(10))

,Работодатель,Кол-во регионов с вакансиями
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
5,Совкомбанк,63
6,МТС,55
7,"ЭФКО, Управляющая компания",49
8,КРОН,48
9,Почта России,48


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [28]:
# текст запроса
query_5_4 = f'''select
                count(distinct e.id)
                from public.employers as e
                left join public.employers_industries as ei on ei.employer_id = e.id
                where ei.industry_id is null
'''

In [29]:
# результат запроса
no_industries = pd.read_sql_query(query_5_4, connection).loc[0, 'count']
print(f"У {no_industries} работодателей не указана сфера деятельности.")

У 8419 работодателей не указана сфера деятельности.


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [30]:
# текст запроса
query_5_5 = f'''select
                e.name
                from public.employers as e
                join public.employers_industries as ei on ei.employer_id = e.id
                group by e.id
                having count(ei.industry_id) = 4
                order by e.name
                offset 2 limit 1
'''

In [31]:
# результат запроса
third_employer = pd.read_sql_query(query_5_5, connection).loc[0, 'name']
print(f"На третьем месте в алфавитном списке (по названию) компаний, у которых \
указано четыре сферы деятельности, находится {third_employer}.")

На третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности, находится 2ГИС.


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [32]:
# текст запроса
query_5_6 = f'''select
                count(ei.employer_id)
                from employers_industries as ei
                join public.employers as e on ei.employer_id = e.id
                join public.industries as i on ei.industry_id = i.id
                where i.name = 'Разработка программного обеспечения'
'''

In [33]:
# результат запроса
sd_employer = pd.read_sql_query(query_5_6, connection).loc[0, 'count']
print(f"У {sd_employer} работодателей в качестве сферы деятельности указана Разработка программного обеспечения.")

У 3553 работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [34]:
# Код для получения списка городов-милионников
import requests # импортируем библиотеку requests
from bs4 import BeautifulSoup # импортируем библиотеку BeautifulSoup

url = 'https://ru.wikipedia.org/wiki/Города-миллионеры_России'
# Выполняем GET-запрос "притворяясь" браузером
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
# Создаём объект BeautifulSoup, указывая html-парсер
page = BeautifulSoup(response.text, 'html.parser')
# Названия городов берём из маркеров на карте, выделенных жирным шрифтом
links = page.find_all('b')
# Полученые объекты объединяем в список
links = [link.text for link in links]
# Из задания 4.1 берём список городов
area_list = area_df['Город'].tolist()
# Сравниваем объекты со страницы со списком городов, чтобы отсеять названия городов
million_cities = []
for link in links:
    if link in area_list:
        million_cities.append(link)
# Выводим получившийся список
print('Города-миллионеры России:', '\n'.join(million_cities), sep='\n')

Города-миллионеры России:
Москва
Санкт-Петербург
Новосибирск
Екатеринбург
Казань
Красноярск
Нижний Новгород
Челябинск
Уфа
Краснодар
Самара
Ростов-на-Дону
Омск
Воронеж
Пермь
Волгоград


In [35]:
# Генерируем фильтр по городам-милионникам, с помощью цикла
filter_area = f'''where e.name = 'Яндекс' and (a.name = '{million_cities[0]}' '''
for i in range(1,len(million_cities)):
    filter_area += f'''or a.name = '{million_cities[i]}' '''
filter_area += ')'

# текст запроса
query_5_7 = f'''-- Основная часть таблицы
                select
                a.name as "Город-миллионник",
                count(v.id)
                from public.vacancies as v
                join public.employers as e on v.employer_id = e.id
                join public.areas as a on v.area_id = a.id            
''' \
                + filter_area \
                + f'''
                group by a.id
                -- Объединяем со строкой "total"
                union
                select
                'total',
                count(v.id)
                from public.vacancies as v
                join public.employers as e on v.employer_id = e.id
                join public.areas as a on v.area_id = a.id
''' \
                + filter_area \
                + f'''
                order by 2
'''

In [36]:
# результат запроса
employer_mc = pd.read_sql_query(query_5_7, connection)
display(employer_mc)

,Город-миллионник,count
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

***Выводы по анализу работодателей:***

На первом месте по количеству вакансий и регионов с публикациями находится компания Яндекс. У данной компании из 1933 вакансий 485 в городах-миллионниках, из них 54 в Москве.

Среди 23501 работодателей у 8419 (36%) не указаны сферы деятельности. Среди тех, у кого указаны, у 3553 (24%) в качестве сферы деятельности указана «Разработка программного обеспечения». Что говорит о том, что у большинства компаний, которые ищут IT-специалистов, сфера деятельности не связана напрямую с разработкой программного обеспечения.

У некоторых работодателей в качестве места регистрации указана страна или регион, а в качестве региона вакансии, как правило, заданы названия населённых пунктов, поэтому регионы вакансии и регистрации работодателя могут не совпадать.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [37]:
# текст запроса
query_6_1 = f'''select
                count(id)
                from public.vacancies
                --удваиваем знак %% для корректной работы запроса в SQLAlchemy
                where lower(name) like '%%data%%' or lower(name) like '%%данн%%'
'''

In [38]:
# результат запроса
ds_vacancies = pd.read_sql_query(query_6_1, connection).loc[0, 'count']
print(f"{ds_vacancies} вакансий имеет отношение к данным.")

1771 вакансий имеет отношение к данным.


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [39]:
# текст запроса
query_6_2 = f'''select
                count(id)
                from public.vacancies
                -- Фильтр для вакансий DS
                where (lower(name) like '%%data scientist%%'
                or lower(name) like '%%data science%%'
                or lower(name) like '%%исследователь данных%%' 
                or (lower(name) like '%%ml%%' and lower(name) not like '%%html%%')
                or lower(name) like '%%machine learning%%'
                or lower(name) like '%%машинн%%обучен%%')
                -- Фильтр для вакансий уровня Junior
                and (lower(name) like '%%junior%%'
                or experience = 'Нет опыта'
                or employment = 'Стажировка')
'''

In [40]:
# результат запроса
j_ds_vacancies = pd.read_sql_query(query_6_2, connection).loc[0, 'count']
print(f"Есть {j_ds_vacancies} подходящяя вакансия для начинающего дата-сайентиста.")

Есть 51 подходящяя вакансия для начинающего дата-сайентиста.


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [41]:
# текст запроса
query_6_3 = f'''select
                count(id)
                from public.vacancies
                -- Фильтр для вакансий DS
                where (lower(name) like '%%data scientist%%'
                or lower(name) like '%%data science%%'
                or lower(name) like '%%исследователь данных%%' 
                or (lower(name) like '%%ml%%' and lower(name) not like '%%html%%')
                or lower(name) like '%%machine learning%%'
                or lower(name) like '%%машинн%%обучен%%')
                -- Фильтр для ключевых навыков
                and (lower(key_skills) like '%%sql%%'
                or lower(key_skills) like '%%postgres%%')
'''

In [42]:
# результат запроса
sql_ds_vacancies = pd.read_sql_query(query_6_3, connection).loc[0, 'count']
print(f"В {sql_ds_vacancies} вакансиях в качестве ключевого навыка указан SQL или postgres.")

В 229 вакансиях в качестве ключевого навыка указан SQL или postgres.


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [43]:
# текст запроса
query_6_3 = f'''select
                count(id)
                from public.vacancies
                -- Фильтр для вакансий DS
                where (lower(name) like '%%data scientist%%'
                or lower(name) like '%%data science%%'
                or lower(name) like '%%исследователь данных%%' 
                or (lower(name) like '%%ml%%' and lower(name) not like '%%html%%')
                or lower(name) like '%%machine learning%%'
                or lower(name) like '%%машинн%%обучен%%')
                -- Фильтр для ключевых навыков
                and (lower(key_skills) like '%%python%%')
'''

In [44]:
# результат запроса
py_ds_vacancies = pd.read_sql_query(query_6_3, connection).loc[0, 'count']
print(f"В {py_ds_vacancies} вакансиях в качестве ключевого навыка указан Python.")

В 357 вакансиях в качестве ключевого навыка указан Python.


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [45]:
# текст запроса
query_6_5 = f'''select
                -- Количество навыков считаем как разность длинны строки и
                -- длинны строки без разделителей + 1 
                avg(length(key_skills) - length(replace(key_skills,CHR(9),''))+1)
                from public.vacancies
                -- Фильтр для вакансий DS
                where (lower(name) like '%%data scientist%%'
                or lower(name) like '%%data science%%'
                or lower(name) like '%%исследователь данных%%' 
                or (name like '%%ML%%' and name not like '%%HTML%%')
                or lower(name) like '%%machine learning%%'
                or lower(name) like '%%машинн%%обучен%%')
                -- Фильтр заполненности поля ключевых навыков
                and key_skills is not null
'''

In [46]:
# результат запроса
ks_ds_vacancies = pd.read_sql_query(query_6_5, connection).loc[0, 'avg']
print(f"{round(ks_ds_vacancies, 2)} ключевых навыков в среднем указывают в вакансиях для DS.")

6.41 ключевых навыков в среднем указывают в вакансиях для DS.


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [47]:
# текст запроса
query_6_6 = f'''select
                -- В числителе сумма значений полей зарплатной вилки,
                -- в знаменателе сумма значений полей в нулевой степени.
                -- Если в знаменателе нет null, то сумма разделится на 2.
                -- Если в знаменателе одно null, то заполненное поле разделится на 1.
                -- Полей с двумя null нет, из-за фильтра.
                -- Результат - среднее полученных частных.
                avg((coalesce(salary_to,0) + coalesce(salary_from,0)) /
                (coalesce(pow(salary_from,0),0) + coalesce(pow(salary_to,0),0)))
                from public.vacancies
                -- Фильтр для вакансий DS
                where (lower(name) like '%%data scientist%%'
                or lower(name) like '%%data science%%'
                or lower(name) like '%%исследователь данных%%' 
                or (name like '%%ML%%' and name not like '%%HTML%%')
                or lower(name) like '%%machine learning%%'
                or lower(name) like '%%машинн%%обучен%%')
                -- Фильтр заполненности полей зарплатной вилки
                and (salary_from is not null or salary_to is not null)
                -- Фильтр опыта работы
                and experience = 'От 3 до 6 лет'
'''


In [48]:
# результат запроса
salary_ds_vacancies = pd.read_sql_query(query_6_6, connection).loc[0, 'avg']
print(f"На зарплату {round(salary_ds_vacancies)} в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет.")

На зарплату 243115 в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет.


***Выводы по предметному анализу:***

Из 49197 вакансий 1771 вакансий имеет отношение к данным, 51 вакансия подходит для начинающего дата-сайентиста.

В среднем 6 ключевых навыков указывают в вакансиях для DS. При этом в 357 вакансиях в качестве ключевого навыка указан Python, а в 229 вакансиях в качестве ключевого навыка указан SQL или postgres.

На зарплату 243 тыс. руб. в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Это превышает среднее значение для верхней границы зарплатной вилки (111 тыс. руб.).

***

# Общий вывод по проекту

Так-как цель создание модели машинного обучения, которая будет рекомендовать вакансии клиентам агентства, претендующим на позицию Data Scientist, подробно рассмотрим вакансии для дата-сайентистов. Дополнительно в фильтре "ML" уберём вакансии содержащие "QML" (C++ разработчик (QML)). Посчитаем число вакансий, удовлетворяющих итоговому условию:

In [49]:
# текст запроса
query_7_0 = f'''select 
                count(id)
                from public.vacancies
                -- Фильтр для вакансий DS
                where (lower(name) like '%%data scientist%%'
                or lower(name) like '%%data science%%'
                or lower(name) like '%%исследователь данных%%' 
                -- Дополнительно исключаем вакансии с QML среди вакансий содержащих ML
                or (name like '%%ML%%' and name not like '%%HTML%%' and name not like '%%QML%%')
                or lower(name) like '%%machine learning%%'
                or lower(name) like '%%машинн%%обучен%%')
'''

In [50]:
# результат запроса
count_ds = pd.read_sql_query(query_7_0, connection).loc[0, 'count']
print(f"{count_ds} вакансий относятся к DS.")

467 вакансий относятся к DS.


1. Рассмотрим заполненность зарплатной вилки в разрезе опыта работы для вакансий DS.

In [51]:
# текст запроса
query_7_1 = f'''select
                -- Создаём таблицу с опытом работы, где хотя бы одно поле зарплатной вилки заполнено
                experience as "Требуемый опыт работы",
                -- Создаём индикатор заполнено/не заполнено для графика
                'Заполнены' as "Заполненность зарплатной вилки",
                count(id) as "Кол-во вакансий"
                from public.vacancies
                -- Фильтр для DS вакансий
                where (lower(name) like '%%data scientist%%'
                or lower(name) like '%%data science%%'
                or lower(name) like '%%исследователь данных%%' 
                or (name like '%%ML%%' and name not like '%%HTML%%' and name not like '%%QML%%')
                or lower(name) like '%%machine learning%%'
                or lower(name) like '%%машинн%%обучен%%')
                and (salary_from is not null or salary_to is not null)
                group by experience
                -- Объединяем с таблицей, где поля не заполнены
                union all
                select
                experience as "Требуемый опыт работы",
                'Не заполнены',
                count(id)
                from public.vacancies
                where (salary_from is null and salary_to is null)
                and (lower(name) like '%%data scientist%%'
                or lower(name) like '%%data science%%'
                or lower(name) like '%%исследователь данных%%' 
                or (name like '%%ML%%' and name not like '%%HTML%%' and name not like '%%QML%%')
                or lower(name) like '%%machine learning%%'
                or lower(name) like '%%машинн%%обучен%%')
                group by experience
'''

In [52]:
# результат запроса
salary_experience = pd.read_sql_query(query_7_1, connection)
display(salary_experience)

# Строим диаграмму "солнечных лучей" по полученным данным
fig = px.sunburst(
    data_frame=salary_experience, # датафрейм
    path=['Требуемый опыт работы', 'Заполненность зарплатной вилки'], # группировка
    values='Кол-во вакансий', # значения
    color_discrete_sequence=['PaleGreen', 'PaleTurquoise', 'Bisque', 'Lavender'], # цвета
    height=800,
    title='Опыт работы и заполненность полей зарплатной вилки' # заголовок
)

fig.update_traces(textinfo="label+percent parent") # добавляем проценты в подписи данных

fig.show()

,Требуемый опыт работы,Заполненность зарплатной вилки,Кол-во вакансий
0,Более 6 лет,Не заполнены,34
1,Нет опыта,Не заполнены,21
2,От 1 года до 3 лет,Не заполнены,192
3,От 3 до 6 лет,Не заполнены,159
4,Нет опыта,Заполнены,7
5,От 1 года до 3 лет,Заполнены,27
6,От 3 до 6 лет,Заполнены,27


2. Определим максимальные и минимальные значения границ зарплатной вилки и количество вакансий DS по регионам вакансии.

In [53]:
# текст запроса
query_7_2 = f'''select
                -- Таблица с регионами, чьё название будет полностью на графике
                a.name as "Регион (исх.)",
                coalesce(min(salary_from),0) || ' - ' || coalesce(max(salary_from),0) as "Зарплата от (min-max)",
                coalesce(min(salary_to),0) || ' - ' || coalesce(max(salary_to),0) as "Зарплата до (min-max)",
                count(v.id) as "Кол-во вакансий",
                -- Добавляем вспомогательный столбец для графика
                a.name as "Регион"
                from public.vacancies as v
                join public.areas as a on v.area_id = a.id 
                -- Фильтр для вакансий DS
                where lower(v.name) like '%%data scientist%%'
                or lower(v.name) like '%%data science%%'
                or lower(v.name) like '%%исследователь данных%%' 
                or (v.name like '%%ML%%' and v.name not like '%%HTML%%' and v.name not like '%%QML%%')
                or lower(v.name) like '%%machine learning%%'
                or lower(v.name) like '%%машинн%%обучен%%'
                group by a.id
                -- Выводим регионы, где вакансий больше 8
                having count(v.id) > 8
                -- Таблица с регионами, которые будут объединены в "Другие" на графике
                union all
                select
                a.name,
                coalesce(min(salary_from),0)::text || ' - ' || coalesce(max(salary_from),0)::text,
                coalesce(min(salary_to),0)::text || ' - ' || coalesce(max(salary_to),0)::text,
                count(v.id) as "Кол-во вакансий",
                -- Во вспомогательном столбце заносим значение "Другие"
                'Другие'
                from public.vacancies as v
                join public.areas as a on v.area_id = a.id 
                where lower(v.name) like '%%data scientist%%'
                or lower(v.name) like '%%data science%%'
                or lower(v.name) like '%%исследователь данных%%' 
                or (v.name like '%%ML%%' and v.name not like '%%HTML%%' and v.name not like '%%QML%%')
                or lower(v.name) like '%%machine learning%%'
                or lower(v.name) like '%%машинн%%обучен%%'
                group by a.id
                -- Выводим регионы, где вакансий меньше 8
                having count(v.id) <= 8
                -- Сортируем по количеству вакансий
                order by 4 desc
'''

In [54]:
# результат запроса
area_and_salary = pd.read_sql_query(query_7_2, connection)
# Выводим данные без вспомогательного столбца
display(area_and_salary.loc[:, 'Регион (исх.)':'Кол-во вакансий'])

# Строим столбчатую диаграмму по полученным данным
fig = px.bar(
    data_frame=area_and_salary.groupby(
        'Регион',
        as_index=False
    )['Кол-во вакансий'].sum(), # датафрейм с группировкой 
    x='Регион', # имена категорий
    y='Кол-во вакансий', # значения
    color='Регион', # разделение по цвету для типа категории
    category_orders={
        'Регион': [
            'Москва', 'Санкт-Петербург', 'Новосибирск', 'Нижний Новгород',
            'Алматы', 'Казань', 'Минск', 'Другие'
        ]
    }, # порядок категорий
    color_discrete_sequence=[
        'PaleGreen', 'PaleTurquoise', 'MediumSeaGreen', 'MediumSlateBlue',
        'BurlyWood', 'Khaki', 'LightBlue', 'Plum'
    ],  # цвета
    text_auto=True, # добавляем значения на график
    title='Вакансии DS по регионам' # заголовок
)

fig.update(layout_showlegend=False) # удаляем легенду

fig.show()

,Регион (исх.),Зарплата от (min-max),Зарплата до (min-max),Кол-во вакансий
0,Москва,45000 - 300000,100000 - 700000,217
1,Санкт-Петербург,50000 - 300000,98000 - 400000,64
2,Новосибирск,60000 - 300000,100000 - 100000,23
3,Нижний Новгород,0 - 0,0 - 0,19
4,Алматы,61024 - 97638,73228 - 146456,16
5,Казань,50000 - 50000,0 - 0,15
6,Минск,0 - 0,0 - 0,9
7,Екатеринбург,100000 - 200000,37000 - 250000,7
8,Томск,0 - 0,0 - 0,7
9,Нур-Султан,0 - 0,0 - 0,6


3. Проанализируем вакансии DS сгруппировав по работодателю.

In [55]:
# текст запроса
query_7_3 = f'''select
                e.name as "Работодатель",
                -- Выводим через ; сферы деятельности из справочника,
                -- если сфер деятельности нет в справочнике выводим пустую строку
                coalesce(string_agg(distinct i.name,'; '),'  ')	 as "Сферы деятельности",
                -- Т.к. на каждую сферу деятельности после соединения будет своя строка,
                -- считаем уникальные id вакансий
                count(distinct v.id) as "Кол-во вакансий",
                count(distinct v.area_id) as "Регионов с вакансиями",
                count(distinct v.name) as "Уникальных вакансий"
                -- Все работодатели есть в справочнике работодателей, данную таблицу присоединяем INNER JOIN
                from public.vacancies as v
                join public.employers as e on v.employer_id = e.id
                -- Чтобы сохранить вакансии работодателей, у которых отсутствует информация о сферах деятельности,
                -- таблицы справочники присоединяем LEFT JOIN
                left join public.employers_industries as ei on ei.employer_id = v.employer_id
                left join public.industries as i on ei.industry_id = i.id
                -- Фильтр для вакансий DS
                where (lower(v.name) like '%%data scientist%%'
                or lower(v.name) like '%%data science%%'
                or lower(v.name) like '%%исследователь данных%%' 
                or (v.name like '%%ML%%' and v.name not like '%%HTML%%' and v.name not like '%%QML%%')
                or lower(v.name) like '%%machine learning%%'
                or lower(v.name) like '%%машинн%%обучен%%')
                group by e.id
                -- Сортируем сначала по кол-ву вакансий, потом по числу уникальных вакансий
                order by count(distinct v.id) desc, count(distinct v.name) desc
'''

In [56]:
# результат запроса
employers_and_vacancies = pd.read_sql_query(query_7_3, connection)
# Выводим общее количество работодателей
print(f"У {employers_and_vacancies.shape[0]} работодателя есть вакансии для DS.")
# Выводим первую 20-ку работодателей по количеству вакансий DS
display(employers_and_vacancies.head(20))

У 221 работодателя есть вакансии для DS.


,Работодатель,Сферы деятельности,Кол-во вакансий,Регионов с вакансиями,Уникальных вакансий
0,СБЕР,Банк,37,3,27
1,Bell Integrator,"Системная интеграция, автоматизации технологи...",25,18,4
2,Банк ВТБ (ПАО),"Банк; Управляющая, инвестиционная компания (уп...",18,9,7
3,VK,"Интернет-компания (поисковики, платежные систе...",15,2,15
4,Positive Technologies,Разработка программного обеспечения; Системная...,11,5,2
5,Яндекс,"Интернет-компания (поисковики, платежные систе...",9,5,5
6,МегаФон,Мобильная связь; Разработка программного обесп...,8,5,5
7,Andersen,Разработка программного обеспечения; Системная...,7,5,3
8,МТС,Мобильная связь,6,1,6
9,Ozon,"Интернет-магазин; Курьерская, почтовая доставк...",6,2,5


4. Дополнительно рассмотрим сферы деятельности работодателей, публикующих DS вакансии.

In [57]:
# текст запроса
query_7_4 = f'''select
                i.name as "Сферы деятельности",
                count(distinct v.employer_id) as "Кол-во работодателей",
                count(distinct v.id) as "Кол-во вакансий работодателей"
                -- Т.к. исследуется информация о сферах деятельности,
                -- работодателей без сферы не учитываем - используем INNER JOIN
                from public.vacancies as v
                join public.employers_industries as ei on ei.employer_id = v.employer_id
                join public.industries as i on ei.industry_id = i.id
                -- Фильтр для вакансий DS
                where (lower(v.name) like '%%data scientist%%'
                or lower(v.name) like '%%data science%%'
                or lower(v.name) like '%%исследователь данных%%' 
                or (v.name like '%%ML%%' and v.name not like '%%HTML%%' and v.name not like '%%QML%%')
                or lower(v.name) like '%%machine learning%%'
                or lower(v.name) like '%%машинн%%обучен%%')
                group by i.name
                -- Сортируем по количеству вакансий
                order by count(distinct v.id) desc
'''

In [58]:
# результат запроса
industries_and_vacancies = pd.read_sql_query(query_7_4, connection)
# Выводим общее количество сфер деятельности
print(f"В {industries_and_vacancies.shape[0]} сверах деятельности работают \
работадатели у которых есть вакансии для DS.")
# Выводим первую 5-ку сфер деятельности для работодателей публикующих вакансий DS
display(industries_and_vacancies.head(5))

В 97 сверах деятельности работают работадатели у которых есть вакансии для DS.


,Сферы деятельности,Кол-во работодателей,Кол-во вакансий работодателей
0,Разработка программного обеспечения,75,145
1,"Системная интеграция, автоматизации технологи...",62,138
2,Банк,23,92
3,"Интернет-компания (поисковики, платежные систе...",37,79
4,Мобильная связь,9,23


5. Выведем топ-10 ключевых навыков, которые наиболее часто указывают в вакансиях DS.

In [59]:
# текст запроса
query_7_5 = f'''select
                key_skills
                from public.vacancies
                -- Фильтр для вакансий DS
                where key_skills is not null
                and (lower(name) like '%%data scientist%%'
                or lower(name) like '%%data science%%'
                or lower(name) like '%%исследователь данных%%' 
                or (name like '%%ML%%' and name not like '%%HTML%%' and name not like '%%QML%%')
                or lower(name) like '%%machine learning%%'
                or lower(name) like '%%машинн%%обучен%%')
'''

In [60]:
# результат запроса
key_skills = pd.read_sql_query(query_7_5, connection)
# Выводим общее количество вакансий с заполненными ключевыми навыками 
print(f"У {key_skills.shape[0]} вакансий DS заполнены ключевые навыки")


# Создаём пустой список, куда будем заносить ключевые навыки
list_key_skills = []
# Добавляем ключевые навыки в список
for i in range(key_skills.shape[0]):
    list_key_skills.extend(key_skills.loc[i,'key_skills'].split('\t'))
# Создаём список уникальных навыков
unic_key_skills = list(set(list_key_skills))
# Создаём пустой список для количества упоминаний уникальных навыков
count_skills = []
# Вычисляем количество упоминаний уникальных навыков
for skill in unic_key_skills:
    count_skills.append(list_key_skills.count(skill))

key_skills_df = pd.DataFrame({
    'Ключевой навык': unic_key_skills,
    'Кол-во упоминаний в вакансиях DS': count_skills
}).sort_values(by='Кол-во упоминаний в вакансиях DS', ascending=False, ignore_index=True)

# Выводим первые 10 навыков по количеству упоминаний
display(key_skills_df.head(10))

# Строим столбчатую диаграмму по полученным данным
fig = px.bar(
    data_frame=key_skills_df.head(10), # датафрейм
    x='Кол-во упоминаний в вакансиях DS', # значения
    y='Ключевой навык', # имена категорий
    color_discrete_sequence=['SteelBlue'],  # цвета
    orientation='h', # горизонтальная ориентация
    labels = {'Ключевой навык':''}, # убираем подпись Y
    text_auto=True, # добавляем значения на график
    title='Топ-10 ключевых навыков по количеству упоминаний в вакансиях DS' # заголовок
)

fig.update_layout(yaxis = {'categoryorder':'total ascending'})
fig.update(layout_showlegend=False) # удаляем легенду

fig.show()


У 418 вакансий DS заполнены ключевые навыки


,Ключевой навык,Кол-во упоминаний в вакансиях DS
0,Python,348
1,SQL,190
2,Machine Learning,114
3,Git,64
4,Математическая статистика,62
5,Data Analysis,54
6,Data Science,52
7,Pandas,52
8,ML,49
9,PyTorch,46


![](plots/experience_ds.png)
![](plots/area_ds.png)
![](plots/key_skills.png)

***Общие выводы:***

Если сравнивать результаты анализа вакансий DS и вакансий IT в целом в разрезе опыта работы и заполненности зарплатной вилки можно отметить следующее:

- У 49% IT-вакансий заполнено хотя бы одно поле зарплатной вилки, у DS-вакансий этот показатель меньше для любой из категорий по опыту работы, а для категории «Более 6 лет» данные по зарплате вообще отсутствуют. Это усложняет подбор вакансий для дата сайентистов по критерию желаемой заработной платы. Возможно будет необходимость строить модель для определения примерной ожидаемой заработной платы для вакансии исходя из региона и опыта работы.
- Соотношение доли категорий по опыту работы также различается. Доля с опытом более 3 лет в целом для IT 32%, для DS - 47%. Доля вакансий без опыта более чем в два раза меньше: IT - 15%, DS - 6%. Это говорит о важности практических навыков и опыта для работодателей при подборе сотрудников на позиции Data Scientist.

Лидеры среди регионов по количеству вакансий для DS примерно те же. Однако из-за того, что вакансии меньше, больше регионов, где вакансий для дата сайентистов нету. И для не готовых к переезду соискателей будет необходимо подбирать предложения или из вакансии в схожих областях (например, связанных с анализом данных) или вакансии с удалённым типом рабочего графика.

Среди сфер деятельности работодателей ищущих специалистов по DS наиболее часто встречаются:

1. Разработка программного обеспечения;
2. Системная интеграция,  автоматизации технологических и бизнес-процессов предприятия, ИТ-консалтинг;
3. Банк;
4. Интернет-компания (поисковики, платежные системы, соц.сети, информационно-познавательные и развлекательные ресурсы, продвижение сайтов и прочее).

Опыт работы соискателя в данных сферах на должностях аналитика или IT-специалиста, возможно, стоит также учитывать и предлагать соответствующие вакансии.

В разрезе работодателей можно отметить, что есть работодатели, которые ищут несколько одинаковых вакансий в разных регионах на удалённую работу (например, Bell Integrator). В будущем необходимо продумать механизм предотвращения вывода подобных дублей для пользователя при запросе.

Самые популярные навыки, которые хотят видеть работодатели у соискателей это: Python (348 упоминаний), SQL (190) и Machine Lerning (114 + 49 (указаны как ML) = 163). То, что навыки в вакансиях могут сокращаться до аббревиатур, стоит также учитывать при подборе рекомендаций клиентам агентства.